In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import gc
from plotly.subplots import make_subplots
from sklearn.model_selection import GridSearchCV
import plotly.graph_objects as go
import pickle

pd.set_option('display.max_rows', 160)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 30)

import warnings
warnings.filterwarnings("ignore")
#dir(pickle)

In [3]:
#data = pd.read_pickle('df_completeMy.pkl')
with open('df_completeMy.pkl', "rb") as fh:
    data = pickle.load(fh)
#del data['subtype']

X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
#free memory
del data
gc.collect();

In [4]:
# Y_train = Y_train.clip(0, 20)
# Y_valid = Y_valid.clip(0, 20)

In [5]:
def build_lgb_model(params, X_train, X_val, y_train, y_val):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    model = lgb.train(params=params, train_set=lgb_train, valid_sets=(lgb_train, lgb_val), verbose_eval=1000)
    #model = lgb.train(params=params, verbose_eval=1000)
    return model

In [12]:
#skip this cell if directly loading saved model 


# params = {
#     "objective" : "regression", 
#     "metric" : "rmse", 
#     'n_estimators':900, 
#     'early_stopping_rounds':50,
#     "num_leaves" : 35, 
#     "learning_rate" : 0.05,   # 0.1
#     "bagging_fraction" : 0.3,
#     "feature_fraction" : 0.5, 
#     "bagging_seed" : 0,
    
#     'max_depth': 13,  
#     'min_child_weight': 0.002, 
#     'min_child_sample': 18,
#     'subsample': 0.8,
#     'colsample_bytree':0.8,
#     'reg_alpha': 0.03,
#     'reg_lambda': 0.03
# }
params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'num_leaves': 1023,
    'min_data_in_leaf':10,
    'feature_fraction':0.7,
    'learning_rate': 0.01,  #0.01
    'num_rounds': 1000,    #1000
    'early_stopping_rounds': 30,
    'seed': 1
}

#designating the categorical features which should be focused on
#cat_features = ['category_id','month','shop_id','shop_city']

lgb_model = build_lgb_model(params, X_train, X_valid, Y_train, Y_valid)

# #save model for later use
# lgb_model.save_model('initial_lgb_model.txt')
Y_test = lgb_model.predict(X_test).clip(0, 20)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.449733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12206
[LightGBM] [Info] Number of data points in the train set: 6115148, number of used features: 74
[LightGBM] [Info] Start training from score 0.292778
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[161]	training's rmse: 0.769556	valid_1's rmse: 0.932361


In [13]:
submission = pd.read_csv('sample_submission.csv')
submission['item_cnt_month'] =lgb_model.predict(X_test).clip(0,20)
submission[['ID', 'item_cnt_month']].to_csv('lightGBM_submission.csv', index=False)